In [ ]:
# import library
from google.colab import files
import pandas as pd
import datetime

# upload file into google colab
uploaded_file = files.upload()
file = next(iter(uploaded_file))


Saving Donor Without Age-2023-07-17-10-17-18.xlsx to Donor Without Age-2023-07-17-10-17-18.xlsx


In [ ]:
# validate national id
def validate_nat_id(national_id):
  # remove empty spaces between digit
  national_id = national_id.replace(" ", "")

  # add hypen to 12 digit number
  if len(national_id) == 14:
    return national_id
  elif len(national_id) == 12:
    national_id = f"{national_id[:6]}-{national_id[6:8]}-{national_id[8:]}"
  else:
    return None

  return national_id

# function to calculate birthdate
def calculate_birthdate(national_id):
  if national_id is None:
    return None

  year = national_id[0:2]
  month = national_id[2:4]
  day = national_id[4:6]

  current_year = datetime.datetime.now().year % 100

  # determine the century based on last 2 digit in year
  if int(year) <= current_year:
    century = 20
  else:
    century = 19

  # check if day between 1-31 only and handle ValueError
  try:
      if int(day) >= 1 and int(day) <= 31:
        birthdate = f"{century}{year}-{month}-{day}"
      else:
        return None
  except ValueError:
      birthdate = None

  return birthdate

# function to calculate age
def calculate_age(birthdate):
  # check birthdate
  if birthdate is None:
    return None

  current_year = datetime.datetime.now().year
  birth_year = int(birthdate[:4])
  age = current_year - birth_year

  return age

In [ ]:
# manipulate file
df = pd.read_excel(file)

def filter_dataframe(df):
    filter_condition = (
        ((df['National ID'].astype(str).str.len() == 12) | (df['National ID'].astype(str).str.len() == 14)) &
        df['National ID'].notna() &
        (df['National ID'] != '')
    )
    filtered_df = df[filter_condition].copy()
    return filtered_df

filter_df = filter_dataframe(df)

filter_df['National ID'] = filter_df['National ID'].apply(validate_nat_id)
filter_df['Birthdate'] = filter_df['National ID'].apply(calculate_birthdate)
filter_df['Age'] = filter_df['Birthdate'].apply(calculate_age)

df.loc[filter_df.index, ['Birthdate', 'Age']] = filter_df[['Birthdate', 'Age']]

df[['National ID', 'Birthdate', 'Age']]


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,National ID,Birthdate,Age
0,930104-06-5310,1993-01-04,30.0
1,630108-02-5199,1963-01-08,60.0
2,630108-02-5199,1963-01-08,60.0
3,790123-03-5692,1979-01-23,44.0
4,770123-14-5033,1977-01-23,46.0
...,...,...,...
690,931228-05-5290,1993-12-28,30.0
691,931228-05-5290,1993-12-28,30.0
692,881205-12-6022,1988-12-05,35.0
693,751261-03-5697,None,NaN


In [ ]:
# rename file with new name
current_filename = file[:-5]
output_file = f'Updated {current_filename}.xlsx'

# save to excel and download
df.to_excel(output_file, index = False)
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>